In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
#|hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [1]:
#|default_exp fastai.learner

In [2]:
#|export
# Python native modules
import os
# Third party libs
from fastcore.all import *
# Local modules

# Learner
> Revised fastai learner that leverages DataPipes for composible and highly flexible training loops.

In [4]:
#|export
class Learner(object): #(dp.iter.IterDataPipe):
    def __init__(self,model,dls,opt,loss_func,cbs,train_loop=None):
        store_attr('model,dls,opt,loss_func')
        self.cbs = L()
        self.add_cbs(cbs)
        self.train_loop = ifnone(train_loop,default_train_loop)
        
    def fit(self,epochs):
        self.it = iter(self.dls[0])
        self.train_pipe = only_train_loop(L(self.it),epochs,self,self.cbs) # Do not pass tuple, otherwise traverse will try to read the dl datapipes
        for res in self.train_pipe:pass

    def add_cbs(self, cbs):
        L(cbs).map(self.add_cb)
        return self

    def remove_cbs(self, cbs):
        L(cbs).map(self.remove_cb)
        return self

    def add_cb(self, cb):
        if isinstance(cb, type): cb = cb()
        cb.learn = self
        # cb.init_pipes()
        setattr(self, cb.name, cb)
        self.cbs.append(cb)
        return self

In [ ]:
# Setup up the core NN
model = DQN(4,2)
# Setup the agent
agent_base = Agent(model,[])
agent = RawOutOfStep(agent_base,agent_base,'state')
agent_ep = DiscreteEpsilonRandomSelect(agent,agent_base,2,min_epsilon=0)
agent = ArgmaxOfStep(agent_ep,agent_base)
agent = ToDiscrete(agent,agent_base)
# Setup the data block 
GymTransformBlock = TransformBlock(
    type_tfms  = GymTypeTransform,
    item_tfms  = (GymStepTransform(agent),DictToTensor),
    batch_tfms = DictCollate,
    cbs = NStepCallback(nsteps=3)
)
# Init with supported loader loop
block = DataBlock(
    blocks=GymTransformBlock,
    loader_loop=simple_iter_loader_loop
)
# Init the loader(s)
dls = block.dataloaders(['CartPole-v1'],n=50,n_workers=0,bs=1)

In [3]:
#|hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
